In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import intel_extension_for_pytorch as ipex  # Import IPEX

# Example dataset class
class BindingAffinityDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        features = self.data[idx]["features"]  # Example features
        label = self.data[idx]["binding_affinity"]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# Simple neural network for demonstration (e.g., for fingerprint or graph features)
class BindingAffinityModel(nn.Module):
    def __init__(self):
        super(BindingAffinityModel, self).__init__()
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess your data
data = [
    {"features": np.random.rand(2048), "binding_affinity": np.random.rand()} for _ in range(1000)
]
dataset = BindingAffinityDataset(data)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Instantiate the model and optimizer
model = BindingAffinityModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Optimize with IPEX by scripting the model
model = torch.jit.script(model)  # Use scripting to prepare for IPEX optimizations
model, optimizer = ipex.optimize(model, optimizer=optimizer)  # Pass both model and optimizer

criterion = nn.MSELoss()  # Loss function for regression

# Simple training loop with IPEX
for epoch in range(10):
    for features, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the trained model in ONNX format with IPEX support
torch.onnx.export(model, torch.randn(1, 2048), "binding_affinity_model_ipex.onnx", input_names=['input'], output_names=['output'])




Epoch 1, Loss: 0.06472690403461456
Epoch 2, Loss: 0.029956413432955742
Epoch 3, Loss: 0.08099202811717987
Epoch 4, Loss: 0.09622466564178467
Epoch 5, Loss: 0.07774963974952698
Epoch 6, Loss: 0.08703283220529556
Epoch 7, Loss: 0.06383894383907318
Epoch 8, Loss: 0.07926104962825775
Epoch 9, Loss: 0.039134036749601364


Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/compile_worker/__main__.py", line 45, in <module>
    main()
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/compile_worker/__main__.py", line 38, in main
    pre_fork_setup()
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/async_compile.py", line 62, in pre_fork_setup
    from triton.compiler.compiler import triton_key
ImportError: cannot import name 'triton_key' from 'triton.compiler.compiler' (/opt/app-root/lib64/python3.9/site-packages/triton/compiler/compiler.py)
W1113 20:25:29.133145 140099703322176 intel_extension_for_pytorch/utils/_logger.py:72] SubprocPool unclean exit


Epoch 10, Loss: 0.028912115842103958


In [2]:
import openvino.runtime as ov

# Create OpenVINO Core object instance
core = ov.Core()

# Define the model directory and name
MODEL_DIR = './'  # Specify your directory as a string
MODEL_NAME = "binding_affinity_model_openvino"

# Read the ONNX model
ov_model = core.read_model("binding_affinity_model_ipex.onnx")

# Compile the model for the specified device
compiled_model = core.compile_model(ov_model, "CPU")

# Save the OpenVINO model to disk in IR format
ov.serialize(ov_model, MODEL_DIR + "/" + f"{MODEL_NAME}.xml")


In [3]:
import openvino.runtime as ov
import numpy as np

# Load the OpenVINO model
core = ov.Core()
model = core.read_model("binding_affinity_model_openvino.xml")
compiled_model = core.compile_model(model, "CPU")

# Prepare a sample input
sample_input = np.random.rand(1, 2048).astype(np.float32)  # Replace with real fingerprint data

# Run inference
output = compiled_model([sample_input])[0]
print("Predicted Binding Affinity:", output[0])


Predicted Binding Affinity: [0.64453125]
